In [1]:
import glob

import pandas as pd
import dask.dataframe as dd

In [2]:
def read_data(location, format, frac=0.02):
    files = glob.glob(f"{location}*.{format}")
    if format == "parquet":
        data = dd.read_parquet(location).sample(frac=0.02)
    # elif format == "h5":
    #     data = dd.concat([read_hdf5(file) for file in files]).sample(frac=0.02)
    data = data.rename(columns={"Distance to CIS": "Distance to CSL"})
    return data

In [3]:
# ddf = read_data("../data/aggregated_data/parquet/", "parquet", frac=1)
# ddf

In [4]:
pd.set_option('display.max_columns', None)
# ddf = dd.read_parquet("../data/aggregated_data/parquet/tickets_full_merge_2024.parquet")
ddf = dd.read_parquet("../data/aggregated_data/parquet/tickets_full_merge_2023.parquet")
ddf.head(5)

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description,Latitude,Longitude,datetime,tempmax,tempmin,temp,conditions,humidity,windspeed,visibility,Closest Middle School,Distance to CMS,Closest High School,Distance to CHS,Closest Individual Landmark,Distance to CIL,Closest Scenic Landmark,Distance to CIS,Closest Business,Industry of CB,Distance to CB
0,1484699750,GCX5397,NY,PAS,1687132800000,63,SUBN,CHEVR,N,30640,13015,28540,20230103,1,1,1,161331,0001,0000,1102A,Manhattan,O,109,SOUTH ST,,0,408,F3,,BBBBBBB,ALL,ALL,WHT,2010,0,,,40.708622,-74.003870,1687132800000,25.7,19.3,21.8,Clear,64.5,21.5,15.9,Spruce Street School,0.303262,Urban Assembly School for Emergency Management...,0.403826,18 Fulton Street Building,0.208080,Bryant Park,5.313856,SEAPORT PARKING LLC,Garage,0.009434
1,1484721329,HEZ7860,NY,PAS,1656892800000,20,SUBN,SUBAR,K,39202,0,0,20240305,122,122,5,160548,0005,0000,0235P,Staten Island,,,MIDLAND BEACH,LOT 8,0,408,C4,,BBBBBBB,ALL,ALL,GL,2016,0,,,40.573161,-74.094586,1656892800000,29.4,19.9,25.0,Rain,42.2,14.0,15.7,I.S. R002 George L. Egbert,0.652695,New Dorp High School,1.187157,Ernest Flagg Estate Cottage: McCall's Demonstr...,1.653102,Coney Island (Riegelman) Boardwalk,9.709113,"BILTMORE GENERAL CONTRACTORS, INC.",Home Improvement Contractor,0.013309
2,1484721330,HDE5505,NY,PAS,1656892800000,20,SUBN,ME/BE,K,39202,0,0,20240303,122,122,5,160548,0005,0000,0156P,Staten Island,,,MIDLAND BEACH,LOT 8,0,408,D,,BBBBBBB,ALL,ALL,WHITE,2019,0,,,40.573161,-74.094586,1656892800000,29.4,19.9,25.0,Rain,42.2,14.0,15.7,I.S. R002 George L. Egbert,0.652695,New Dorp High School,1.187157,Ernest Flagg Estate Cottage: McCall's Demonstr...,1.653102,Coney Island (Riegelman) Boardwalk,9.709113,"BILTMORE GENERAL CONTRACTORS, INC.",Home Improvement Contractor,0.013309
3,1484724136,JEB6269,NY,PAS,1656892800000,68,SDN,CHEVR,N,22278,0,0,20221018,122,122,5,160690,0005,0000,0626P,Staten Island,,,CEDAR GROVE BEACH,PARKING LOT,0,408,F3,,BBBBBBB,ALL,ALL,SILVE,2015,0,,,40.558055,-74.101515,1656892800000,29.4,19.9,25.0,Rain,42.2,14.0,15.7,I.S. R002 George L. Egbert,2.077059,New Dorp High School,1.365887,Gustave A. Mayer House,2.571409,Coney Island (Riegelman) Boardwalk,10.431480,U AND P DELI & GROCERY INC,Electronic Cigarette Dealer,0.430214
4,1484725888,KET8159,NY,PAS,1657324800000,20,SDN,CHEVR,K,42850,33720,61830,20221029,84,84,4,160448,0004,0000,0841P,Brooklyn,F,60,FURMAN ST,,0,408,C4,,BBBBBBB,ALL,ALL,SILVE,2011,0,,,40.697978,-73.993521,1657324800000,29.3,22.8,25.1,"Rain, Partially cloudy",61.5,21.6,15.9,Dock Street School for STEAM Studies,0.576071,"Urban Assembly School for Law and Justice, The",0.537792,Brooklyn Trust Company Building,0.398727,Prospect Park,4.541993,SUN NEWS INC.,Tobacco Retail Dealer,0.058913


In [5]:
# ddf[ddf["Issue Date"] == 1717027200000].head(5)

In [6]:
daily_ddf = ddf.groupby("Issue Date").agg({"tempmax": "first", "tempmin": "first", "temp": "first", "conditions": "first", "humidity": "first", "windspeed": "first", "visibility": "first"})
daily_ddf["count"] = ddf.groupby("Issue Date").size()
daily_ddf = daily_ddf.dropna()
daily_ddf.compute()

,tempmax,tempmin,temp,conditions,humidity,windspeed,visibility,count
Issue Date,,,,,,,,
1687132800000,25.7,19.3,21.8,Clear,64.5,21.5,15.9,12192
1656892800000,29.4,19.9,25.0,Rain,42.2,14.0,15.7,3059
1657324800000,29.3,22.8,25.1,"Rain, Partially cloudy",61.5,21.6,15.9,12824
1656806400000,31.0,24.3,26.8,Partially cloudy,52.4,21.5,16.0,5147
1656720000000,31.1,24.3,27.3,"Rain, Partially cloudy",66.9,16.2,16.0,17168
...,...,...,...,...,...,...,...,...
1685836800000,21.1,11.6,16.0,"Rain, Partially cloudy",58.2,26.9,15.9,7546
1686182400000,22.1,14.3,18.3,Partially cloudy,51.5,14.6,10.7,23819
1686528000000,23.9,19.4,21.3,"Rain, Partially cloudy",77.2,23.7,14.1,25963


# ML

In [7]:
from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import StandardScaler, Categorizer, LabelEncoder
from dask_ml.metrics import mean_squared_error

In [8]:
# ce = Categorizer(columns=["conditions"])
# daily_ddf = ce.fit_transform(daily_ddf)
le = LabelEncoder()
le.fit(daily_ddf["conditions"])
daily_ddf["conditions"] = le.transform(daily_ddf["conditions"])

X, y = daily_ddf.drop(columns=["count"]), daily_ddf["count"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

c:\Users\Luka\miniconda3\envs\BD_project\lib\site-packages\dask\dataframe\core.py:3769: UserWarning: Dask currently has limited support for converting pandas extension dtypes to arrays. Converting string to object dtype.
  warnings.warn(
c:\Users\Luka\miniconda3\envs\BD_project\lib\site-packages\dask\dataframe\core.py:3769: UserWarning: Dask currently has limited support for converting pandas extension dtypes to arrays. Converting string to object dtype.
  warnings.warn(


In [9]:
display(X_train.compute(), y_train.compute())

,tempmax,tempmin,temp,conditions,humidity,windspeed,visibility
Issue Date,,,,,,,
1683331200000,22.8,10.6,15.8,2,58.2,17.8,15.9
1657929600000,28.8,22.8,24.8,5,76.5,16.1,15.8
1665187200000,15.6,9.3,12.2,5,53.3,25.9,16.0
1673568000000,13.2,3.9,9.4,5,75.8,23.5,14.8
1672876800000,9.4,7.1,8.5,4,90.7,15.2,9.6
...,...,...,...,...,...,...,...
1678320000000,12.1,1.3,5.9,3,53.2,20.1,16.0
1665532800000,20.8,14.5,17.2,0,54.4,31.7,16.0
1657843200000,28.9,23.4,25.8,2,56.9,17.8,16.0


Issue Date
1683331200000    17467
1657929600000    16174
1665187200000    17730
1673568000000    26648
1672876800000    26113
                 ...  
1678320000000    34150
1665532800000    29335
1657843200000    31606
1660694400000    27651
1657670400000    27564
Name: count, Length: 290, dtype: int64

In [10]:
X_train_, y_train_ = X_train.values.persist(), y_train.values.persist()
X_test_, y_test_ = X_test.values.persist(), y_test.values.persist()

In [11]:
X_train_.compute_chunk_sizes(), y_train_.compute_chunk_sizes(), X_test_.compute_chunk_sizes(), y_test_.compute_chunk_sizes()

(dask.array<values, shape=(290, 7), dtype=float64, chunksize=(290, 7), chunktype=numpy.ndarray>,
 dask.array<values, shape=(290,), dtype=int64, chunksize=(290,), chunktype=numpy.ndarray>,
 dask.array<values, shape=(73, 7), dtype=float64, chunksize=(73, 7), chunktype=numpy.ndarray>,
 dask.array<values, shape=(73,), dtype=int64, chunksize=(73,), chunktype=numpy.ndarray>)

## Dask ML

In [12]:
from dask_ml.linear_model import LinearRegression

In [13]:
lr = LinearRegression()
lr.fit(X_train_, y_train_)

LinearRegression()

In [14]:
preds = lr.predict(X_test_)

In [15]:
y_test_.compute(), preds.compute()

(array([34826, 24818, 19997, 29557, 30262, 16081, 31260,  5575, 28365,
        18241, 32281, 17527,  5625, 32084, 16706, 32834, 32546, 33168,
        26487, 16415,  5147, 24572, 27014,  6566, 26689, 31096, 30154,
        17601, 28181, 19409, 30122, 28448, 14657, 16317,  5936, 12824,
        28251, 26601, 30016, 29402, 22142, 15879, 31068,  6300, 28272,
        13157, 26091,  4174, 23345, 31990, 31016, 30400, 30391, 28996,
        32247, 24019,  6185, 26586, 28424,  7122,  7043, 26800, 16781,
        32560, 18752, 29984, 23523,  5466, 28562, 32272, 30509, 15575,
        30849]),
 array([22081.16373914, 18665.29071249, 20702.52122236, 22824.41902836,
        22830.55402312, 22364.25426133, 21125.85114841, 22658.0989063 ,
        22319.87624325, 21917.89527741, 21714.87486408, 20368.676844  ,
        21139.39366401, 21762.01555337, 22439.00906773, 21151.59291815,
        22513.73951582, 23212.99526006, 22338.03869081, 22841.9003793 ,
        21832.92057914, 21281.07043482, 21378.3113864 ,

In [16]:
mean_squared_error(y_test_, preds, squared=False)

np.float64(9201.268214669191)

## XGBoost

In [17]:
import xgboost as xgb

In [28]:
clf = xgb.XGBRegressor( 
    n_estimators=1000,
    max_depth=10,
    learning_rate=0.3,
    n_jobs=-1,
    random_state=42
)
clf.fit(X_train_, y_train_)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [29]:
preds = clf.predict(X_test_)

In [30]:
y_test_.compute(), preds

(array([34826, 24818, 19997, 29557, 30262, 16081, 31260,  5575, 28365,
        18241, 32281, 17527,  5625, 32084, 16706, 32834, 32546, 33168,
        26487, 16415,  5147, 24572, 27014,  6566, 26689, 31096, 30154,
        17601, 28181, 19409, 30122, 28448, 14657, 16317,  5936, 12824,
        28251, 26601, 30016, 29402, 22142, 15879, 31068,  6300, 28272,
        13157, 26091,  4174, 23345, 31990, 31016, 30400, 30391, 28996,
        32247, 24019,  6185, 26586, 28424,  7122,  7043, 26800, 16781,
        32560, 18752, 29984, 23523,  5466, 28562, 32272, 30509, 15575,
        30849]),
 array([30146.19 , 22701.938, 27058.346, 31998.518, 22428.166, 23246.6  ,
        17426.209, 19489.63 , 18236.057, 21369.283, 20803.887, 18707.328,
        26950.342, 23331.07 , 15157.346, 22510.76 , 23894.008, 11353.484,
        22749.328, 16263.862, 20850.633, 25325.883, 24190.27 , 24765.887,
        15123.416, 18575.217, 24554.152, 25835.662, 22154.805, 23020.29 ,
        24955.393, 18730.578, 26694.559, 2375

In [31]:
mean_squared_error(y_test_, preds, squared=False)

np.float64(10429.182524727936)

## Scikit

In [32]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor

In [38]:
batch_size = 10
X_batches = [X_train_[i:i+batch_size] for i in range(0, len(X_train_), batch_size)]
y_batches = [y_train_[i:i+batch_size] for i in range(0, len(y_train_), batch_size)]

In [53]:
clf = SGDRegressor(
    max_iter=1000,
    tol=1e-3,
    random_state=42
)
for X_batch, y_batch in zip(X_batches, y_batches):
    clf.partial_fit(X_batch, y_batch)

In [50]:
preds = clf.predict(X_test_)

In [51]:
y_test_.compute(), preds

(array([34826, 24818, 19997, 29557, 30262, 16081, 31260,  5575, 28365,
        18241, 32281, 17527,  5625, 32084, 16706, 32834, 32546, 33168,
        26487, 16415,  5147, 24572, 27014,  6566, 26689, 31096, 30154,
        17601, 28181, 19409, 30122, 28448, 14657, 16317,  5936, 12824,
        28251, 26601, 30016, 29402, 22142, 15879, 31068,  6300, 28272,
        13157, 26091,  4174, 23345, 31990, 31016, 30400, 30391, 28996,
        32247, 24019,  6185, 26586, 28424,  7122,  7043, 26800, 16781,
        32560, 18752, 29984, 23523,  5466, 28562, 32272, 30509, 15575,
        30849]),
 array([-1.31370716e+12, -4.17843834e+12, -1.55893675e+12, -9.10892036e+12,
        -8.78917038e+12, -5.92355563e+12, -1.94237183e+12, -8.78322065e+12,
        -9.24620699e+12, -2.22469665e+12, -6.58955677e+12, -2.22134909e+12,
        -4.15658064e+12, -4.83068428e+12, -8.32288923e+12, -1.99424201e+12,
        -3.08342508e+12, -7.68834223e+12, -1.06015229e+13, -8.38369521e+12,
        -8.58316499e+12, -2.7821689

In [52]:
mean_squared_error(y_test_, preds, squared=False)

np.float64(5906698590324.957)